<a href="https://colab.research.google.com/github/itzelmor02/Aprendizaje-de-Patrones-IMG/blob/main/Practica05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Realiza una regresión polinomial del conjunto de entrenamiento probando distintos grados
de polinomio. Calcula el sesgo y varianza para cada grado utilizado. Imprime
los resultados en un dataframe de pandas: grado del polinomio, sesgo, varianza, MAE,
MSE, R2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Definimos scale = 7.53
scale = 7.53

# Generamos las muestras aleatorias de Rayleigh
sample_size = 10000
rayleigh_samples = np.random.rayleigh(scale, size=sample_size)

# Definimos los puntos para el histograma
num_bins = 160

# Creamos el histograma
hist, bins = np.histogram(rayleigh_samples, bins=num_bins, density=True)

# Obtener puntos como valores x
x_values = (bins[:-1] + bins[1:]) / 2

# Dividimos los datos en conjuntos de entrenamiento y de prueba (80-20)
x_train, x_test, y_train, y_test = train_test_split(x_values, hist, test_size=0.2, random_state=42)

# Dividimos el conjunto de entrenamiento en conjuntos de entrenamiento y validación (80-20)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Definimos los grados de polinomios que vamos a usar
degrees = [1, 2, 3, 4, 5]

# Creamos listas para guardar los resultados
results = []

# Iteramos sobre cada grado
for degree in degrees:
    # Transformamos las características a características polinomiales
    poly_features = PolynomialFeatures(degree=degree)
    X_train_poly = poly_features.fit_transform(x_train.reshape(-1, 1))
    X_val_poly = poly_features.transform(x_val.reshape(-1, 1))

    # Ajustamos el modelo de regresión lineal
    model = LinearRegression()
    model.fit(X_train_poly, y_train)

    # Predecimos en el conjunto de validación
    val_predictions = model.predict(X_val_poly)

    # Calculamos bias^2
    bias_sq = np.mean((val_predictions - y_val)**2)

    # Calculamos variance
    variance = np.var(val_predictions)

    # Calculamos MAE
    mae = mean_absolute_error(y_val, val_predictions)

    # Calculamos MSE
    mse = mean_squared_error(y_val, val_predictions)

    # Calculamos R^2
    r2 = r2_score(y_val, val_predictions)

    # Agregamos resultados a la lista
    results.append([degree, bias_sq, variance, mae, mse, r2])

# Creamos el dataframe de pandas
results_df = pd.DataFrame(results, columns=['Degree', 'Bias^2', 'Variance', 'MAE', 'MSE', 'R2'])
print(results_df)


   Degree    Bias^2  Variance       MAE       MSE        R2
0       1  0.000264  0.000383  0.013164  0.000264  0.699030
1       2  0.000279  0.000524  0.014843  0.000279  0.681133
2       3  0.000064  0.000854  0.007015  0.000064  0.926990
3       4  0.000012  0.000859  0.002683  0.000012  0.986053
4       5  0.000012  0.000867  0.002810  0.000012  0.986101


Repetimos el análisis usando regularización y validación cruzada

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Definimos scale = 7.53
scale = 7.53

# Generamos las muestras aleatorias de Rayleigh
sample_size = 10000
rayleigh_samples = np.random.rayleigh(scale, size=sample_size)

# Definimos los puntos para el histograma
num_bins = 160

# Creamos el histograma
hist, bins = np.histogram(rayleigh_samples, bins=num_bins, density=True)

# Obtener puntos como valores x
x_values = (bins[:-1] + bins[1:]) / 2

# Dividimos los datos en conjuntos de entrenamiento y de prueba (80-20)
x_train, x_test, y_train, y_test = train_test_split(x_values, hist, test_size=0.2, random_state=42)

# Dividimos el conjunto de entrenamiento en conjuntos de entrenamiento y validación (80-20)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Definimos los grados de polinomios que vamos a usar
degrees = [1, 2, 3, 4, 5]

# Creamos listas para guardar los resultados
results = []

# Iteramos sobre cada grado
for degree in degrees:
    # Creamos una tubería con características polinómicas y regresión de crestas con validación cruzada
    model = make_pipeline(PolynomialFeatures(degree), StandardScaler(), RidgeCV(alphas=np.logspace(-6, 6, 13)))

    # Ajustamos el modelo
    model.fit(x_train.reshape(-1, 1), y_train)

    # Predecimos en el conjunto de validación
    val_predictions = model.predict(x_val.reshape(-1, 1))

    # Calculamos MAE
    mae = mean_absolute_error(y_val, val_predictions)

    # Calculamos MSE
    mse = mean_squared_error(y_val, val_predictions)

    # Calculamos R^2
    r2 = r2_score(y_val, val_predictions)

    # Usamos validación cruzada para obtener el promedio de R^2
    cv_scores = cross_val_score(model, x_train.reshape(-1, 1), y_train, cv=5, scoring='r2')
    avg_cv_r2 = np.mean(cv_scores)

    # Agregamos los resultados a la lista
    results.append([degree, mae, mse, r2, avg_cv_r2])

# Creamos el dataframe de pandas
results_df = pd.DataFrame(results, columns=['Degree', 'MAE', 'MSE', 'R2', 'CV R2'])
print(results_df)

   Degree       MAE       MSE        R2     CV R2
0       1  0.020586  0.000570  0.417507  0.379902
1       2  0.014849  0.000304  0.689084  0.537440
2       3  0.006853  0.000063  0.935840  0.902751
3       4  0.002702  0.000016  0.984025  0.981129
4       5  0.003136  0.000016  0.983866  0.981909


**Mejor modelo polinomial**
El primer modelo porque muestra un bias ^2, varianza, MAE y MSE más bajos, junto con valores R2 más altos, lo que indica un mejor rendimiento del modelo en términos de ajuste a los datos y precisión de la predicción.

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Generamos muestras aleatorias a partir de la distribución de Rayleigh utilizando el parámetro de escala MLE
sample_size = 10000
rayleigh_samples_mle = np.random.rayleigh(mle_scale, size=sample_size)

# Damos las métricas de rendimiento
polynomial_results = {
    'Degree': [1, 2, 3, 4, 5],
    'Bias^2': [0.000264, 0.000279, 0.000064, 0.000012, 0.000012],
    'Variance': [0.000383, 0.000524, 0.000854, 0.000859, 0.000867],
    'MAE': [0.013164, 0.014843, 0.007015, 0.002683, 0.002810],
    'MSE': [0.000264, 0.000279, 0.000064, 0.000012, 0.000012],
    'R2': [0.699030, 0.681133, 0.926990, 0.986053, 0.986101]
}

# Calculamos métricas de rendimiento para el modelo de distribución de Rayleigh utilizando el parámetro de escala MLE
mle_mae = mean_absolute_error(rayleigh_samples_mle, np.zeros_like(rayleigh_samples_mle))
mle_mse = mean_squared_error(rayleigh_samples_mle, np.zeros_like(rayleigh_samples_mle))
mle_r2 = r2_score(rayleigh_samples_mle, np.zeros_like(rayleigh_samples_mle))

# Imprimimos las métricas de rendimiento para comparar
print("Mejor modelo:")
print("Degree    MAE       MSE        R2")
for i in range(len(polynomial_results['Degree'])):
    print("{}       {:.6f}  {:.6f}  {:.6f}".format(polynomial_results['Degree'][i],
                                                    polynomial_results['MAE'][i],
                                                    polynomial_results['MSE'][i],
                                                    polynomial_results['R2'][i]))

print("\nRendimiento Modelo de Distribución de Rayleigh(MLE):")
print("MAE:", mle_mae)
print("MSE:", mle_mse)
print("R2:", mle_r2)

Mejor modelo:
Degree    MAE       MSE        R2
1       0.013164  0.000264  0.699030
2       0.014843  0.000279  0.681133
3       0.007015  0.000064  0.926990
4       0.002683  0.000012  0.986053
5       0.002810  0.000012  0.986101

Rendimiento Modelo de Distribución de Rayleigh(MLE):
MAE: 4.887027457522986
MSE: 30.480070858475013
R2: -3.6202692338033255
